# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.4252783542292127)),
 (1, np.float64(1.4252783542292127)),
 (2, np.float64(1.4252783542292127)),
 (3, np.float64(1.4252783542292127)),
 (4, np.float64(1.4252783542292127))]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18), ('what', 10)]),
 (1, [('banana', 2), ('is', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.016396805280677107)),
   (None, np.float64(0.08452144270908424)),
   (None, np.float64(0.1683264774310388)),
   (None, np.float64(0.22634870570719567)),
   (None, np.float64(0.24362853213705704)),
   (None, np.float64(0.2601424495080158)),
   (None, np.float64(0.31249360501168666)),
   (None, np.float64(0.329390081193483)),
   (None, np.float64(0.36013756569922695)),
   (None, np.float64(0.37013306315022854)),
   (None, np.float64(0.39637560398907357)),
   (None, np.float64(0.4998221850489355))]),
 (1,
  [(None, np.float64(0.5138903368170532)),
   (None, np.float64(0.5245627077361705)),
   (None, np.float64(0.5278224875900452)),
   (None, np.float64(0.580938157900022)),
   (None, np.float64(0.6405148920841304)),
   (None, np.float64(0.6421324250261455)),
   (None, np.float64(0.6597346202276771)),
   (None, np.float64(0.7066422627666414)),
   (None, np.float64(0.7599322832467811)),
   (None, np.float64(0.7622342309495449)),
   (None, np.float64(0.771703791888

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
data = [3, 7, 2, 9, 4, 9, 1]

def RECORDREADER():
    for i, x in enumerate(data):
        yield (i, x)

def MAP(k1, v1):
    yield ("max", v1)

def REDUCE(key, values):
    yield (key, max(values))

list(MapReduce(RECORDREADER, MAP, REDUCE))


[('max', 9)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
data = [3, 7, 2, 9, 4, 9, 1]

def RECORDREADER():
    for i, x in enumerate(data):
        yield (i, x)

def MAP(k1, v1):
    yield ("avg", (v1, 1))

def REDUCE(key, values):
    s = 0
    c = 0
    for v, cnt in values:
        s += v
        c += cnt
    yield (key, s / c)

list(MapReduce(RECORDREADER, MAP, REDUCE))


[('avg', 5.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
import random
random.seed(42)

def groupbykey_sorted(iterable):
    data = sorted(iterable, key=lambda x: x[0])
    result = []
    current_key = None
    current_values = []
    for k, v in data:
        if k != current_key:
            if current_key is not None:
                result.append((current_key, current_values))
            current_key = k
            current_values = [v]
        else:
            current_values.append(v)
    if current_key is not None:
        result.append((current_key, current_values))
    return result

test = [(random.randint(1,5), random.randint(-100,100)) for _ in range(30)]
groupbykey_sorted(test)


[(1, [-94, 73, -93, -45, 94, -3, -9, -4, 41]),
 (2, [-65, 29, 14, 78, 95]),
 (3, [-38, -61, -74, 54, -89, 60]),
 (4, [-13, 37]),
 (5, [-78, 8, -94, -50, 7, -29, -8, -51])]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
data = [
    (1, "A"),
    (2, "B"),
    (1, "A"),
    (3, "C"),
    (2, "B"),
    (4, "D"),
    (3, "C"),
    (5, "E"),
]

def RECORDREADER():
    for i, x in enumerate(data):
        yield (i, x)

def MAP(k1, v1):
    yield (v1, 1)

def REDUCE(key, values):
    yield key

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(1, 'A'), (2, 'B'), (3, 'C'), (4, 'D'), (5, 'E')]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
R = [
    (1, "Alice", 23),
    (2, "Bob", 17),
    (3, "Charlie", 35),
    (4, "Diana", 19),
    (5, "Eve", 42)
]
def RECORDREADER():
    for t in R:
        yield (t, t)

def MAP(k1, v1):
    # C: возраст >= 21
    if v1[2] >= 21:
        yield (v1, v1)

def REDUCE(key, values):
    # identity function
    for v in values:
        yield (key, v)

list(MapReduce(RECORDREADER, MAP, REDUCE))

[((1, 'Alice', 23), (1, 'Alice', 23)),
 ((3, 'Charlie', 35), (3, 'Charlie', 35)),
 ((5, 'Eve', 42), (5, 'Eve', 42))]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
R = [
    (1, "Alice", "IT", 5000),
    (2, "Bob", "HR", 4000),
    (3, "Charlie", "IT", 6000),
    (4, "Diana", "Finance", 7000),
    (5, "Eve", "IT", 5000),
]
def RECORDREADER():
    for t in R:
        yield (t, t)

def MAP(k1, v1):
    # формируем t' оставляя только атрибуты из S
    t_prime = (v1[2], v1[3])   # department, salary
    yield (t_prime, t_prime)

def REDUCE(key, values):
    # вернуть одну пару (t', t')
    yield (key, key)

list(MapReduce(RECORDREADER, MAP, REDUCE))

[(('IT', 5000), ('IT', 5000)),
 (('HR', 4000), ('HR', 4000)),
 (('IT', 6000), ('IT', 6000)),
 (('Finance', 7000), ('Finance', 7000))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
R = [
    (1, "IT", 5000),
    (2, "HR", 4000),
    (3, "Finance", 7000),
    (4, "IT", 6000),
    (2, "HR", 4000)
]

S = [
    (3, "Finance", 7000),
    (5, "Marketing", 4500),
    (6, "IT", 5000),
    (1, "IT", 5000),
    (7, "HR", 3800)
]

def RECORDREADER():
    for t in R + S:
        yield (t, t)

def MAP(k1, v1):
    yield (v1, v1)

def REDUCE(key, values):
    yield (key, key)

list(MapReduce(RECORDREADER, MAP, REDUCE))


[((1, 'IT', 5000), (1, 'IT', 5000)),
 ((2, 'HR', 4000), (2, 'HR', 4000)),
 ((3, 'Finance', 7000), (3, 'Finance', 7000)),
 ((4, 'IT', 6000), (4, 'IT', 6000)),
 ((5, 'Marketing', 4500), (5, 'Marketing', 4500)),
 ((6, 'IT', 5000), (6, 'IT', 5000)),
 ((7, 'HR', 3800), (7, 'HR', 3800))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
R = [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie"),
    (4, "Diana"),
    (2, "Bob"),          # дубликат в R
    (5, "Eve")
]

S = [
    (3, "Charlie"),
    (6, "Frank"),
    (2, "Bob"),
    (7, "Grace"),
    (3, "Charlie"),      # дубликат в S
    (8, "Helen")
]

def RECORDREADER():
    for t in R:
        yield (t, "R")
    for t in S:
        yield (t, "S")

def MAP(k1, v1):
    yield (k1, v1)

def REDUCE(key, values):
    vals = list(values)
    if "R" in vals and "S" in vals:
        yield (key, key)

list(MapReduce(RECORDREADER, MAP, REDUCE))


[((2, 'Bob'), (2, 'Bob')), ((3, 'Charlie'), (3, 'Charlie'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
R = [
    (1, "IT", 5000),
    (2, "HR", 4000),
    (3, "Finance", 7000),
    (4, "IT", 6000),
    (2, "HR", 4000)
]

S = [
    (3, "Finance", 7000),
    (5, "Marketing", 4500),
    (6, "IT", 5000),
    (1, "IT", 5000),
    (7, "HR", 3800)
]

def RECORDREADER():
    for t in R:
        yield (t, "R")
    for t in S:
        yield (t, "S")

def MAP(k1, v1):
    yield (k1, v1)

def REDUCE(key, values):
    vals = list(values)
    if vals == ["R"]:
        yield (key, key)

list(MapReduce(RECORDREADER, MAP, REDUCE))


[((4, 'IT', 6000), (4, 'IT', 6000))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
R = [
    (101, 1),
    (102, 2),
    (103, 3),
    (104, 4),
    (105, 2),   # дубликат ключа 2 в R
    (106, 5),
    (107, 6),
    (108, 6),   # дубликат ключа 6 в R
    (109, 7),
    (110, 8),
    (111, 9),   # ключ есть в R, но нет в S
    (112, 10),
    (113, 11),  # ключ есть в R, но нет в S
    (114, 3),   # еще один дубликат ключа 3 в R
    (115, 12),
]

S = [
    (1,  "Alice"),
    (2,  "Bob"),
    (2,  "Bobby"),     # дубликат ключа 2 в S (даст декартово произведение с R по ключу 2)
    (3,  "Charlie"),
    (3,  "Charles"),   # дубликат/конфликт по ключу 3
    (4,  "Diana"),
    (5,  "Eve"),
    (6,  "Frank"),
    (6,  "Franklin"),  # дубликат/альтернативное имя по ключу 6
    (7,  "Grace"),
    (8,  "Heidi"),
    (10, "Judy"),
    (12, "Oscar"),
    (13, "Peggy"),     # ключ есть только в S (не сджойнится)
]

def RECORDREADER():
    for a,b in R:
        yield (b, ("R", a))
    for b,c in S:
        yield (b, ("S", c))

def MAP(k1, v1):
    yield (k1, v1)

def REDUCE(key, values):
    vals = list(values)
    R_vals = [v for tag,v in vals if tag=="R"]
    S_vals = [v for tag,v in vals if tag=="S"]
    for a in R_vals:
        for c in S_vals:
            yield ((a,key,c), (a,key,c))

list(MapReduce(RECORDREADER, MAP, REDUCE))


[((101, 1, 'Alice'), (101, 1, 'Alice')),
 ((102, 2, 'Bob'), (102, 2, 'Bob')),
 ((102, 2, 'Bobby'), (102, 2, 'Bobby')),
 ((105, 2, 'Bob'), (105, 2, 'Bob')),
 ((105, 2, 'Bobby'), (105, 2, 'Bobby')),
 ((103, 3, 'Charlie'), (103, 3, 'Charlie')),
 ((103, 3, 'Charles'), (103, 3, 'Charles')),
 ((114, 3, 'Charlie'), (114, 3, 'Charlie')),
 ((114, 3, 'Charles'), (114, 3, 'Charles')),
 ((104, 4, 'Diana'), (104, 4, 'Diana')),
 ((106, 5, 'Eve'), (106, 5, 'Eve')),
 ((107, 6, 'Frank'), (107, 6, 'Frank')),
 ((107, 6, 'Franklin'), (107, 6, 'Franklin')),
 ((108, 6, 'Frank'), (108, 6, 'Frank')),
 ((108, 6, 'Franklin'), (108, 6, 'Franklin')),
 ((109, 7, 'Grace'), (109, 7, 'Grace')),
 ((110, 8, 'Heidi'), (110, 8, 'Heidi')),
 ((112, 10, 'Judy'), (112, 10, 'Judy')),
 ((115, 12, 'Oscar'), (115, 12, 'Oscar'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
data = [
    (1, 10),
    (1, 5),
    (1, -3),
    (1, 0),

    (2, 7),
    (2, 13),
    (2, -2),

    (3, 100),     # ключ встречается 1 раз
    (4, 8),
    (4, 8),       # дубликат значения
    (4, 1),

    (5, 1000),
    (5, -250),
    (5, 30),

    (6, 2),
    (6, 2),
    (6, 2),
    (6, 2),       # много одинаковых

    (7, 999999),
    (7, 1),

    (8, -10),     # только отрицательные
    (8, -20),
    (8, -30),

    (9, 42),
    (10, 0),      # ключ с нулём
]
def RECORDREADER():
    for a,b in data:
        yield (a,b)

def MAP(k1, v1):
    yield (k1, v1)

def REDUCE(key, values):
    yield (key, sum(values))

list(MapReduce(RECORDREADER, MAP, REDUCE))


[(1, 12),
 (2, 18),
 (3, 100),
 (4, 17),
 (5, 780),
 (6, 8),
 (7, 1000000),
 (8, -60),
 (9, 42),
 (10, 0)]

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [32]:
import numpy as np

matrix = np.ones((3, 3))
vector = np.array([1, 0, 0, 0])

maps = 2
reducers = 1

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for i in range(split.shape[0]):
            for j in range(split.shape[1]):
                yield ((i, j), (split[i, j], vector[j], split.shape[1]))

    split_size = int(np.ceil(len(matrix) / maps))

    for i in range(0, len(matrix), split_size):
        yield RECORDREADER(matrix[i: i + split_size])


def MAP(coordinates: [int, int], values: [int, int, int]):
    i, j = coordinates
    matrix_value, vector_value, cols = values
    yield ((i, cols), matrix_value * vector_value)


def REDUCE(keys: [int, int], products: Iterator[NamedTuple]):
    i, cols = keys
    yield (i, sum(products) / (len(products) / cols))


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

9 key-value pairs were sent over a network.


[(0, [(0, np.float64(1.0)), (1, np.float64(1.0))])]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [33]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [34]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])


def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(I):
        yield ((i, k), small_mat[i, j] * w)


def REDUCE(key, values):
    (i, k) = key
    yield (i, k), sum(values)

Проверьте своё решение

In [35]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [36]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [37]:
I = 3
J = 4
K = 5 * 10


small_mat = np.random.rand(I, J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield (((i, j), small_mat[i, j]),((j, k), big_mat[j, k]))

def MAP(list_1, list_2):
  (k1, v1) = list_1
  (k2, v2) = list_2
  yield ((k1[0], k2[1]), v1 * v2)

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

In [38]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [39]:
I = 3
J = 4
K = 5*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
  def RECORDREADER(key, split):
    mat = []
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        mat.append(((key, i, j), split[i, j]))
    return mat

  yield RECORDREADER('S', small_mat)
  yield RECORDREADER('B', big_mat)


def MAP(k, v):
  (mat, i, j) = k
  w = v
  if mat == 'S':
    yield (j, (mat, i, w))
  else:
    yield (i, (mat, j, w))

def REDUCE(_, values):
  small = [v for v in values if v[0] == 'S']
  big = [v for v in values if v[0] == 'B']
  for s in small:
    for b in big:
      yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
    for v in p[1]:
        solution.append(v)

212 key-value pairs were sent over a network.
600 key-value pairs were sent over a network.


In [40]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [41]:
I = 3
J = 4
K = 5*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
    global maps
    def RECORDREADER(key, split):
        mat = []
        for i in range(split.shape[0]):
            for j in range(split.shape[1]):
                mat.append(((key, i, j), split[i, j]))
        return mat

    small = RECORDREADER('S', small_mat)
    split_size =  int(np.ceil(len(small)/maps))
    for i in range(0, len(small), split_size):
        yield small[i:i+split_size]

    big = RECORDREADER('B', big_mat)
    split_size =  int(np.ceil(len(big)/maps))
    for i in range(0, len(big), split_size):
      yield big[i:i+split_size]

def MAP(k, v):
    (mat, i, j) = k
    w = v
    if mat == 'S':
        yield (j, (mat, i, w))
    else:
        yield (i, (mat, j, w))

def REDUCE(key, values):
    small = [v for v in values if v[0] == 'S']
    big = [v for v in values if v[0] == 'B']
    for s in small:
        for b in big:
          yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
    for j in joined:
        yield j[1]

def MAP_MUL(k1, v1):
    yield (k1, v1)

def REDUCE_MUL(key, values):
    res_val = 0
    for v in values:
      res_val += v
    yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
    for v in p[1]:
      solution.append(v)

212 key-value pairs were sent over a network.
600 key-value pairs were sent over a network.
